# Todos:
- Build Beta prior distributions for shooting percentages on position by position basis.
- Build Gamma prior for each poisson distribution on position by position basis.
- Store the Beta and Gamma distribution for each player so that it is easy to update by exploiting conjugate prior property
- The positional Beta and Gamma priors should apply to each player at the beginning of their career.
- The Beta and Gamma priors should be adjusted to increase the variance at the beginning of each season due to the uncertainty of what happened during the offseason. (Note: for Gamma(a,b), $E[G] = \frac{a}{b}$ and $Var[G] = \frac{a}{b^2}$, so we can increase the variance while preserving the mean by multiplying $a$ and $b$ by some value $\lambda\in(0,1)$. Similarly for Beta(a,b), $E[B] = \frac{a}{b}$ and $Var[B] = \frac{ab}{(a+b)^2(a+b+1)}$. Multiplying $a$ and $b$ by $\lambda$ will preserve $E[B]$, but the Variance will increase since the denominator will be multiplied by $O(\lambda^2)$ and the numerator by $O(\lambda)$ making the denominator decrease by more than the numerator, and hence, increasing the variance.
- Write script to update and store these distributions for each statistic, for each player.
- Write script to fetch distributions to make a projection

In [1]:
%matplotlib inline
import player as data_pl
import team as data_te
import game as data_ga
import utils as anal_ut
import multiprocessing as mp
import numpy as np
import scipy
import scipy.stats as scst
import datetime
from numpy import linalg
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0)
import matplotlib.colors as colors
import pandas as pd
import time
from sklearn import cluster
from sklearn import linear_model
from collections import Counter
import math
import statsmodels.api as sm
import urllib2
import re
import os
import unicodedata

In [7]:
## Converts Names from FanDuel's Player Sheet to Look Up-able names with the NBA API
FDPL2NBA = {
    'Brad Beal': 'Bradley Beal',
    'C.J. McCollum': 'CJ McCollum',
    'J.J. Redick': 'JJ Redick',
    'C.J. Miles': 'CJ Miles',
    'P.J. Tucker': 'PJ Tucker',
    'Nene Hilario': 'Nene',
    'Patrick Mills': 'Patty Mills',
    'T.J. Warren': 'TJ Warren',
    'Luc Richard Mbah a Moute': 'Luc Mbah a Moute',
    'C.J. Watson': 'CJ Watson',
    'C.J. Wilcox': 'CJ Wilcox',
    'Roy Devyn Marble': 'Devyn Marble',
    'K.J. McDaniels': 'KJ McDaniels',
    'T.J. McConnell': 'TJ McConnell',
    'Chuck Hayes': 'Charles Hayes',
    'Joseph Young': 'Joe Young',
    'Glenn Robinson III': 'Glenn Robinson',
    'Louis Amundson': 'Lou Amundson',
    
}

We can have a CSV file with rows of player parameters and other values.

Columns:
- Name
- Player ID (output of get_player(fname, lname))
- Position
- Current team
- For FG2%, FG3%, FT%, the Beta $\alpha$ and $\beta$ parameters
- For FG2A, FG3A, FTA, AST, REB, BLK, STL, TOV, the Gamma parameters $\alpha$ and $\beta$
- datetime of last update

# General Beta and Gamma Priors
Compute the p and lambda values for each player at each position that has played at least 50 minutes(?) in their career and compute the mean and variance. These will be used to set the alpha and beta in the Beta and Gamma distributions using the method of moments. This should only have to be run once per season.

In [5]:
pl = data_pl.PlayerList(season='2015-16', only_current=1).info()
table = pd.DataFrame()
MIN_THRESHOLD = 50
for name in pl.DISPLAY_LAST_COMMA_FIRST:
    try:
        pid = pl[pl.DISPLAY_LAST_COMMA_FIRST.str.lower() == name.lower()]['PERSON_ID'].tolist()[0]
        gl = data_pl.PlayerGameLogs(pid, season='2014-15').info()
    except Exception, exc:
        print exc.message
        print name
        continue
    mins = float(np.sum(gl.MIN))
    if mins < MIN_THRESHOLD:
        continue
    rates = {}
    for st in ('AST','BLK','STL','TOV','FG2A','FG3A','FTA','REB'):
        if st == 'FG2A':
            rates[st] = np.sum(gl.FGA-gl.FG3A) / mins
        else:
            rates[st] = np.sum(gl[st]) / mins
    rates['FG2%'] = np.sum(gl.FGM-gl.FG3M) / float(np.sum(gl.FGA-gl.FG3A)) if float(np.sum(gl.FGA))>0 else 0
    rates['FG3%'] = np.sum(gl.FG3M) / float(np.sum(gl.FG3A)) if float(np.sum(gl.FG3A))>0 else 0
    rates['FT%'] = np.sum(gl.FTM) / float(np.sum(gl.FTA)) if float(np.sum(gl.FTA))>0 else 0
    table = table.append(pd.DataFrame(rates, index=[pid]))#, ignore_index=True)
table

AST       BLK      FG2%      FG2A      FG3%      FG3A       FT%  \
203112  0.052795  0.017081  0.494465  0.210404  0.300000  0.046584  0.783505   
203919  0.064516  0.028226  0.409836  0.245968  0.400000  0.100806  0.608696   
203500  0.037162  0.048423  0.546599  0.223536  0.000000  0.001126  0.502439   
201167  0.051559  0.002798  0.466425  0.220224  0.354354  0.133094  0.843434   
201582  0.049215  0.053403  0.550152  0.344503  0.000000  0.000000  0.818182   
202332  0.076766  0.066530  0.478405  0.308086  0.000000  0.000000  0.781250   
200746  0.049324  0.027049  0.474809  0.521082  0.352381  0.041766  0.845304   
202730  0.068480  0.039400  0.471572  0.280488  0.000000  0.000000  0.702128   
2754    0.052216  0.018215  0.504695  0.258652  0.344828  0.017608  0.626984   
202329  0.043129  0.045322  0.484979  0.170322  0.274194  0.090643  0.711864   
200811  0.074786  0.055556  0.429134  0.271368  0.000000  0.001068  0.471264   
2365    0.038019  0.053935  0.597938  0.171530  0.307692  0.011494  0.666667   
101187  0.047619  0.002869  0.530435  0.131956  0.347619  0.120482  0.811881   
203937  0.079096  0.019774  0.358974  0.220339  0.272727  0.031073  0.642857   
201583  0.032797  0.011926  0.456757  0.220632  0.339833  0.214073  0.854305   
203507  0.081432  0.033438  0.510870  0.289536  0.159091  0.017309  0.740634   
2546    0.085315  0.011888  0.473684  0.438462  0.340782  0.125175  0.797468   
201202  0.012315  0.120690  0.580645  0.152709  0.000000  0.000000  0.681818   
2772    0.071185  0.005790  0.484507  0.120913  0.349550  0.189033  0.853147   
201589  0.057809  0.026369  0.470803  0.277890  0.236364  0.111562  0.779661   
201600  0.036400  0.027300  0.516990  0.208291  0.000000  0.000000  0.582222   
201571  0.179735  0.001527  0.436409  0.204175  0.337398  0.125255  0.868293   
202337  0.029021  0.013301  0.428571  0.093108  0.513043  0.139057  0.684211   
203946  0.015152  0.015152  0.214286  0.212121  0.000000  0.000000  0.800000   
2571    0.100712  0.008138  0.510949  0.278739  0.383929  0.113937  0.783784   
200826  0.192251  0.000731  0.463415  0.269737  0.323353  0.122076  0.808511   
200745  0.058599  0.034395  0.465625  0.407643  0.365854  0.052229  0.813084   
203084  0.050065  0.008200  0.519274  0.190332  0.404651  0.092792  0.720000   
2440    0.050132  0.021988  0.568548  0.109059  0.361702  0.165347  0.778947   
203115  0.082485  0.019348  0.473881  0.272912  0.270588  0.086558  0.787234   
...          ...       ...       ...       ...       ...       ...       ...   
202349  0.151759  0.005025  0.435829  0.187940  0.378917  0.176382  0.757576   
101111  0.028065  0.032496  0.475177  0.208272  0.375566  0.326440  0.571429   
203943  0.015564  0.035019  0.396825  0.245136  0.384615  0.050584  0.692308   
202696  0.058149  0.021361  0.524167  0.474684  0.333333  0.002373  0.752294   
2548    0.151469  0.010638  0.488798  0.497467  0.284314  0.051672  0.767568   
203079  0.073923  0.009524  0.431429  0.317460  0.296748  0.111565  0.680233   
202689  0.150142  0.014636  0.417974  0.330973  0.303571  0.132200  0.826855   
202322  0.278775  0.015839  0.478398  0.334037  0.299539  0.076382  0.784530   
203933  0.040584  0.014610  0.559585  0.313312  0.238095  0.034091  0.736842   
201228  0.146582  0.006342  0.458678  0.170543  0.400000  0.123326  0.826347   
101110  0.048571  0.002857  0.291667  0.137143  0.232558  0.122857  0.750000   
2561    0.117616  0.025316  0.478979  0.351266  0.200000  0.010549  0.739130   
201566  0.249348  0.006082  0.457312  0.513901  0.298611  0.125109  0.834862   
202355  0.005245  0.107517  0.627907  0.338287  0.000000  0.000000  0.500000   
203963  0.056075  0.018692  0.500000  0.336449  0.166667  0.056075  0.782609   
203952  0.057220  0.016829  0.453017  0.340289  0.309524  0.042410  0.759657   
203912  0.079208  0.009901  0.473684  0.188119  0.368421  0.188119  1.000000   
101114  0.212021  0.008045  0.395131  0.252721  0.367089  0.112163  0.834025   
202682  0.034

In [6]:
mean = np.mean(table, axis=0)
var = np.var(table, axis=0, ddof=1)

res = pd.DataFrame([mean, var], index=['mean','var'])

# MoM: E[p] = a/(a+b) => a = bE[p]/(1-E[p]) = b*m
# Var[p] = ab/( (a+b)^2 * (a+b+1) )
# Var[p] = mb^2 / ( b^2(m+1)^2 * (m*b+b+1) )
def B_beta(b, m, v):
    return m*b**2 / ( b**2 * (m+1)**2 * (b*m+b+1) ) - v

alphas = []
betas = []
# Beta
for st in ('FG2%','FG3%','FT%'):
    m, v = tuple(res[st].tolist())
    mp = m/(1-m)
    beta = scipy.optimize.fsolve(B_beta, .1, args=(mp, v), maxfev=1000)[0]
    alpha = mp*beta
    alphas.append(alpha)
    betas.append(beta)

# Gamma
for st in ('AST','BLK','REB','TOV','STL','FG2A','FG3A','FTA'):
    m, v = res[st].tolist()
    beta = m/v
    alpha = m*beta
    alphas.append(alpha)
    betas.append(beta)
    
naive_beta_gamma_params = pd.DataFrame({'alpha':alphas, 'beta':betas}, index=('FG2%','FG3%','FT%','AST','BLK','REB',
                                                    'TOV','STL','FG2A','FG3A','FTA'))
naive_beta_gamma_params

alpha        beta
FG2%  31.035937   33.919900
FG3%   2.443612    6.409595
FT%    8.949248    3.203207
AST    2.252913   26.268113
BLK    1.020570   49.956805
REB    4.621319   25.836819
TOV    6.804641  121.335172
STL    6.206485  195.646862
FG2A   6.814011   27.906805
FG3A   1.689129   18.179037
FTA    3.702938   40.379696

# Create Beta and Gamma's for Each Player
Start each player with the naive_beta_gamma_params in their FROM_YEAR and update over one season. Then, at the beginning of each season, multiply all the $\alpha$ and $\beta$ by $\lambda\in(0,1)$ to preserve the expectation but inflate the variance. Repeat until current date.

The value of $\lambda$ to use for each of the distributions could be set by running a backtest and finding the value which has the lowest prediction error.

In [7]:
lm = .8
player_distributions = {}
for row_ind in xrange(len(pl.PERSON_ID)):
    row = pl.ix[row_ind]
    pid = row.PERSON_ID
    dists = naive_beta_gamma_params.copy()
    print '%3d: %d' % (row_ind, pid)
    
    for year in xrange(int(row.FROM_YEAR), int(row.TO_YEAR)+1):
        dists *= lm
        
        season = '%d-%02d' % (year, year-2000+1 if year >= 2000 else (year - 1900 + 1) % 100)
        gl = data_pl.PlayerGameLogs(pid, season=season).info()
        
        mins = float(np.sum(gl.MIN))
        cts = {}
        for st in ('AST','BLK','STL','TOV','FG2A','FG2M','FG3A','FG3M','FTA','FTM','REB'):
            if st == 'FG2A':
                cts[st] = np.sum(gl.FGA-gl.FG3A)
            elif st == 'FG2M':
                cts[st] = np.sum(gl.FGM-gl.FG3M)
            else:
                cts[st] = np.sum(gl[st])
        
        a = [cts['AST'], cts['BLK'], cts['STL'], cts['TOV'], cts['REB'],
             cts['FG2M'],             cts['FG3M'],             cts['FTM'],
             cts['FG2A'], cts['FG3A'], cts['FTA']]
        b = [mins,       mins,       mins,       mins,       mins,       
             cts['FG2A']-cts['FG2M'], cts['FG3A']-cts['FG3M'], cts['FTA']-cts['FTM'], 
             mins,        mins,        mins]
        tmp = pd.DataFrame({'alpha':a, 'beta':b}, 
                    index=('AST','BLK','STL','TOV','REB','FG2%','FG3%','FT%','FG2A','FG3A','FTA'))
        dists += tmp
    player_distributions[pid] = dists.copy()

player_distributions

  0: 203112
  1: 203919
  2: 203500
  3: 201167
  4: 201582
  5: 202332
  6: 200746
  7: 1626146
  8: 202730
  9: 2754
 10: 202329
 11: 200811
 12: 2365
 13: 101187
 14: 202341
 15: 1626147
 16: 203937
 17: 201583
 18: 203507
 19: 2546
 20: 201202
 21: 2772
 22: 201589
 23: 201600
 24: 201571
 25: 202337
 26: 203946
 27: 2571
 28: 200826
 29: 200745
 30: 203084
 31: 2440
 32: 203115
 33: 101138
 34: 201587
 35: 201573
 36: 203382
 37: 203145
 38: 203078
 39: 201158
 40: 203461
 41: 201976
 42: 202687
 43: 202357
 44: 204028
 45: 201971
 46: 2581
 47: 202339
 48: 202711
 49: 101106
 50: 2588
 51: 1626164
 52: 202344
 53: 2547
 54: 202340
 55: 201147
 56: 201166
 57: 1626148
 58: 203900
 59: 977
 60: 201978
 61: 203493
 62: 203504
 63: 202692
 64: 2406
 65: 202710
 66: 2446
 67: 203998
 68: 101181
 69: 203484
 70: 203477
 71: 203991
 72: 201960
 73: 1713
 74: 203487
 75: 201956
 76: 1626161
 77: 201596
 78: 2199
 79: 201163
 80: 1626176
 81: 203546
 82: 203903
 83: 202708
 84: 201954
 85

{708:             alpha         beta
 FG2%  1417.187923  1397.145339
 FG3%    12.388044    44.240245
 FT%    612.121113   141.312266
 AST    676.130103  7254.015160
 BLK    239.716867  7254.233650
 REB   2127.607124  7254.011182
 TOV    416.422596  7254.891999
 STL    275.430619  7255.577403
 FG2A  2813.796999  7254.030274
 FG3A    56.562213  7253.940551
 FTA    753.355446  7254.145316, 977:             alpha         beta
 FG2%  1694.260986  1853.619188
 FG3%   337.165538   716.063642
 FT%   1425.005967   278.166086
 AST   1147.512047  8218.460989
 BLK     75.258523  8218.734101
 REB   1209.550871  8218.456017
 TOV    730.098895  8219.557038
 STL    312.809520  8220.413793
 FG2A  3547.209844  8218.479882
 FG3A  1053.146583  8218.367728
 FTA   1703.074637  8218.623685, 1495:             alpha         beta
 FG2%  1899.549808  1856.619180
 FG3%     4.598397    22.425610
 FT%   1025.288228   397.395441
 AST    859.714490  8997.902825
 BLK    586.413912  8998.244215
 REB   2902.842992  8997

# Plot Distributions for Fun

In [5]:
dist = player_distributions[data_pl.get_player('Stephen','Curry', season='2015-16', just_id=True).tolist()[0]]
print dist

for st in ('AST','BLK','REB','TOV','STL','FG2A','FG3A','FTA'):
    x = np.linspace(0,1,100)
    a = dist.alpha.loc[st]
    b = dist.beta.loc[st]
    na = naive_beta_gamma_params.alpha[st]
    nb = naive_beta_gamma_params.beta[st]
    
    plt.plot(x, scst.gamma.pdf(x, na, scale=1./nb), 'r--', label='Naive Prior')
    plt.plot(x, scst.gamma.pdf(x, a, scale=1./b), 'b--', label='Posterior')
    plt.title(st)
    plt.grid()
    plt.legend(loc='best')
    plt.show()

for st in ('FG2%','FG3%','FT%'):
    x = np.linspace(0,1,100)
    a = dist.alpha.loc[st]
    b = dist.beta.loc[st]
    na = naive_beta_gamma_params.alpha[st]
    nb = naive_beta_gamma_params.beta[st]
    
    plt.plot(x, scst.beta.pdf(x, na, nb), 'r--', label='Naive Prior')
    plt.plot(x, scst.beta.pdf(x, a, b), 'b--', label='Posterior')
    plt.title(st)
    plt.grid()
    plt.legend(loc='best')
    plt.show()

NameError: name 'player_distributions' is not defined

# Write Distribution Parameters to File

In [8]:
today = datetime.datetime.now().date()
filename = 'Parameters/player_parameters_%s.csv' % today

with open(filename, 'w') as writer:
    writer.write('PERSON_ID,%s' % ','.join(['%s_%s' % (param, st) for param in ('alpha','beta')
                for st in ('AST','BLK','STL','TOV','REB','FG2%','FG3%','FT%','FG2A','FG3A','FTA')]))
    writer.write('\n')
    for row_ind in xrange(len(pl.PERSON_ID)):
        pid = pl.PERSON_ID.ix[row_ind]
        writer.write('%d,' % pid)
        
        for param in ('alpha','beta'):
            for st in ('AST','BLK','STL','TOV','REB','FG2%','FG3%','FT%','FG2A','FG3A','FTA'):
                v = player_distributions[pid][param].loc[st]
                writer.write('%f,' % v)
        writer.write('\n')

# Read Distribution Parameters from File

In [6]:
today = datetime.datetime.now().date() - datetime.timedelta(days=1)
filename = 'Parameters/player_parameters_%s.csv' % today

player_params = pd.read_csv(filename, index_col=False)
player_params = player_params.set_index('PERSON_ID')
player_params

alpha_AST   alpha_BLK  alpha_STL   alpha_TOV  alpha_OREB  \
PERSON_ID                                                              
203112      25.099371   10.420361  11.570021   22.164406   31.613440   
203919      21.261001    8.022084  25.225571   22.824432   10.717975   
203500      29.060009   45.977284  14.371012   34.009478   76.684036   
201167      36.389566    1.906461   5.494064   24.246763    7.754399   
201582      25.057089   29.934875  15.077046   40.303089   61.483859   
202332      30.811752   38.319219  18.751634   35.105065   54.598611   
200746      42.682918   24.243423  11.741429   39.573562   65.815210   
1626146      2.261001    3.022084   6.225571    7.824432    3.717975   
202730      47.443748   24.341385  11.591787   29.701559   73.618371   
2754        38.418174   11.599054  57.241224   36.235872   49.023102   
202329      49.163455   30.094987  27.488349   47.543001   51.906018   
200811      32.831769   33.328231  19.842318   37.798110   72.617797   
2365        14.245565   24.323444   9.702845   16.844341   31.638623   
101187      17.501669    1.244177  11.401037   12.863143    6.824636   
202341      31.394739    5.559748  15.906091   26.603882   12.366378   
1626147      5.261001    3.022084   8.225571   13.824432    3.717975   
203937      40.644916   13.798376  25.987772   21.257993    8.693199   
201583      33.854828   12.344524  16.990543   41.314441   61.637297   
203507      67.269852   27.188428  25.850632   59.820079   33.288786   
2546        85.234804   14.737262  27.559607   59.369030   41.551417   
201202      12.568923   71.100047  20.612128   25.523230   66.579377   
2772        54.844762    6.022849  47.157811   47.132483   20.108201   
201589      38.242374   24.758480  27.559550   31.391036   46.552707   
201600      12.606138   14.587424   7.390707   28.345818   50.640432   
201571     103.738253    1.548152  18.984644   47.649442    7.559418   
202337      29.401284   10.365109  11.064347   22.174680   15.166659   
203946       3.261001    2.022084   8.225571    9.824432    8.717975   
2571        77.754222    7.611339  35.954165   39.942587   17.618935   
200826     139.224473    1.220297  13.714964   51.382324   15.072191   
200745      25.354227   19.884930   3.800582   35.553232   30.532397   
...               ...         ...        ...         ...         ...   
202689     135.186833   17.969389  42.294361   50.285358   15.702124   
202322     199.190732   19.266095  45.313520   95.622184   15.288249   
203933      23.276462    6.880885  20.798168   30.472938   41.804209   
201228      78.055189    2.484551  24.109008   31.381443    9.286430   
101110      28.662177    3.005836  11.840202   27.484980   14.176818   
201603      36.997237    1.635679  12.360854   31.856908   11.304165   
2561        69.136258   14.200947  15.965342   44.957361   29.790065   
201566     254.584775    8.097665  63.156682  132.991959   53.049907   
202355       6.166346  108.456840  19.499713   50.310603   88.670798   
203963       8.261001    3.022084  11.225571   13.824432   11.717975   
203952      45.394244   15.152011  20.312579   50.561875   41.704272   
203912      10.261001    2.022084   9.225571   16.824432    2.717975   
101114     162.231836    6.755633  34.201858   57.384482    6.500733   
202682      23.464580    5.733425  17.890189   25.717592   19.433368   
101150      64.443957    4.725892  24.943129   39.619692    8.000116   
101107      40.237446   26.321874  22.818823   25.568116   36.052508   
2590       139.050649    6.429629  17.652518   63.581031    7.535693   
1626159     23.261001    4.022084  15.225571   29.824432   16.717975   
203481      24.359038   46.312865  17.282320   26.730801   53.179873   
1626174      5.261001    6.022084  10.225571    8.824432   10.717975   
1897        37.057023   15.543856  40.115731   29.987058   30.533341   
201148      15.028649   34.311311  13.820868   13.819514   40.319199   
1626153      2.261001    1.022084   6.225571    7.82

# Update Parameters

In [7]:
date = today.strftime('%b %d, %Y').upper()
print date
for pid in player_params.index:
    try:
        gl = data_pl.PlayerGameLogs(pid, season='2015-16').info()
        gl = gl[gl.GAME_DATE == date]
        if len(gl.GAME_DATE) == 0:
            continue
        gl = gl.ix[gl.index.tolist()[0]]
    except KeyError:
        continue
    mins = gl.MIN
    for st in ('AST','OREB','DREB','BLK','STL','TOV','FG2A','FG3A','FTA'):
        if st == 'FG2A':
            player_params['alpha_FG2A'].loc[pid] += gl.FGA - gl.FG3A
            player_params['beta_FG2A'].loc[pid] += mins
            player_params['alpha_FG2%'].loc[pid] += gl.FGM - gl.FG3M
            player_params['beta_FG2%'].loc[pid] += gl.FGA - gl.FG3A - (gl.FGM - gl.FG3M)
        else:
            player_params['alpha_%s' % st].loc[pid] += gl[st]
            player_params['beta_%s' % st].loc[pid] += mins
        if st in ('FG3A','FTA'):
            s = st[:-1]
            player_params['alpha_%s%%' % s].loc[pid] += gl['%sM' % s]
            player_params['beta_%s%%' % s].loc[pid] += gl[st] - gl['%sM' % s]
filename = 'Parameters/player_parameters_%s.csv' % (today+datetime.timedelta(days=1))
player_params.to_csv(filename)
player_params

NOV 29, 2015


alpha_AST   alpha_BLK  alpha_STL   alpha_TOV  alpha_OREB  \
PERSON_ID                                                              
203112      25.099371   10.420361  11.570021   22.164406   31.613440   
203919      21.261001    8.022084  25.225571   22.824432   10.717975   
203500      29.060009   45.977284  14.371012   34.009478   76.684036   
201167      40.389566    1.906461   6.494064   25.246763    8.754399   
201582      25.057089   29.934875  15.077046   40.303089   61.483859   
202332      30.811752   38.319219  18.751634   35.105065   54.598611   
200746      42.682918   24.243423  11.741429   39.573562   65.815210   
1626146      2.261001    3.022084   6.225571    7.824432    3.717975   
202730      47.443748   24.341385  11.591787   29.701559   74.618371   
2754        38.418174   11.599054  59.241224   36.235872   49.023102   
202329      49.163455   30.094987  27.488349   47.543001   51.906018   
200811      32.831769   33.328231  19.842318   37.798110   72.617797   
2365        14.245565   24.323444   9.702845   16.844341   31.638623   
101187      17.501669    1.244177  11.401037   12.863143    6.824636   
202341      31.394739    5.559748  15.906091   26.603882   12.366378   
1626147      5.261001    3.022084   8.225571   13.824432    3.717975   
203937      40.644916   13.798376  25.987772   21.257993    8.693199   
201583      33.854828   12.344524  16.990543   41.314441   61.637297   
203507      70.269852   28.188428  26.850632   59.820079   33.288786   
2546        85.234804   14.737262  27.559607   59.369030   41.551417   
201202      12.568923   71.100047  20.612128   25.523230   66.579377   
2772        55.844762    6.022849  49.157811   47.132483   23.108201   
201589      38.242374   24.758480  27.559550   31.391036   46.552707   
201600      12.606138   14.587424   7.390707   28.345818   50.640432   
201571     103.738253    1.548152  18.984644   47.649442    7.559418   
202337      29.401284   10.365109  11.064347   22.174680   15.166659   
203946       3.261001    2.022084   8.225571    9.824432    8.717975   
2571        77.754222    7.611339  35.954165   39.942587   17.618935   
200826     139.224473    1.220297  13.714964   51.382324   15.072191   
200745      25.354227   19.884930   3.800582   35.553232   30.532397   
...               ...         ...        ...         ...         ...   
202689     138.186833   18.969389  45.294361   52.285358   16.702124   
202322     199.190732   19.266095  45.313520   95.622184   15.288249   
203933      23.276462    9.880885  21.798168   30.472938   44.804209   
201228      78.055189    2.484551  24.109008   31.381443    9.286430   
101110      28.662177    3.005836  11.840202   27.484980   14.176818   
201603      36.997237    1.635679  12.360854   31.856908   11.304165   
2561        69.136258   14.200947  15.965342   44.957361   29.790065   
201566     254.584775    8.097665  63.156682  132.991959   53.049907   
202355       6.166346  108.456840  19.499713   50.310603   88.670798   
203963       8.261001    3.022084  11.225571   13.824432   11.717975   
203952      47.394244   16.152011  21.312579   54.561875   41.704272   
203912      10.261001    2.022084   9.225571   16.824432    2.717975   
101114     162.231836    6.755633  34.201858   57.384482    6.500733   
202682      24.464580    5.733425  17.890189   26.717592   19.433368   
101150      64.443957    4.725892  24.943129   39.619692    8.000116   
101107      42.237446   27.321874  22.818823   25.568116   37.052508   
2590       139.050649    6.429629  17.652518   63.581031    7.535693   
1626159     23.261001    4.022084  15.225571   29.824432   16.717975   
203481      24.359038   46.312865  17.282320   26.730801   53.179873   
1626174      5.261001    6.022084  10.225571    8.824432   10.717975   
1897        38.057023   15.543856  42.115731   29.987058   32.533341   
201148      15.028649   34.311311  13.820868   13.819514   40.319199   
1626153      2.261001    1.022084   6.225571    7.82